## Hardware Testbed and Large-scale Testbed Co-simulation

## Co-Simulation

In [1]:
# --- imports ---

import os
import subprocess

import time
import logging
logger = logging.getLogger(__name__)

import csv
import time, datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import andes
andes.config_logger(30)
print(andes.__version__)

%matplotlib inline

# --- set logging level ---
# logger.setLevel(logging.WARNING)
from core_io import data_read, data_write, io_config, cs_stat, cs_config, cs_col

1.8.7.post5+gb9281204


In [2]:
cs_config['t_total'] = 5000

In [3]:
andes.config_logger(stream_level=20)

In [4]:

is_test = False
rflie = 'datar.txt'
wfile = 'dataw.txt'
if is_test:
    rflie = 't' + rflie

### Co-Simulation

In [5]:

# --- Pre Process ---
# --- set path ---
path_proj = os.getcwd()
path_case = os.path.join(path_proj, 'case')
path_data = os.path.join(path_proj, 'data')
path_ltb = os.path.join(path_proj, 'core_ltb.py')
filer = os.path.join(path_data, rflie)
filew = os.path.join(path_data, wfile)
path_ehtb = os.path.join(path_data, 'ehtb.py')
path_out = os.path.join(os.path.abspath('..'), 'output')

case1 = os.path.join(path_case, 'ieee14_htb.xlsx')
case2 = os.path.join(path_case, 'pjm5_htb.xlsx')
case3 = os.path.join(path_case, 'npcc_htb.xlsx')
case4 = os.path.join(path_case, 'ieee39_htb.xlsx')

# --- set case ---
ss = andes.load(case4,
                no_output=True,
                default_config=False,
                setup=False)

#  --- HTB variables ---
pq_htb = 'PQ_2'  # load represents for HTB
bus_htb = ss.PQ.get(idx=pq_htb, src='bus', attr='v')  # get bus index of HTB
bus_slack = ss.Slack.bus.v[0]  # get bus index of slack bus

# add Bus Freq. Measurement to HTB bus
ss.add('BusFreq', {'idx': 'BusFreq_HTB',
                   'name': 'BusFreq_HTB',
                   'bus': bus_htb,
                   'Tf': 0.02,
                   'Tw': 0.02,
                   'fn': 60})
ss.add('BusFreq', {'idx': 'BusFreq_Slack',
                   'name': 'BusFreq_Slack',
                   'bus': bus_slack,
                   'Tf': 0.02,
                   'Tw': 0.02,
                   'fn': 60})

# ss.add('Output', {'model': 'GENCLS',
#                    'varname': 'omega'})
# ss.add('Output', {'model': 'GENROU',
#                    'varname': 'omega'})
# ss.add('Output', {'model': 'Bus',
#                    'varname': 'a'})
# ss.add('Output', {'model': 'Bus',
#                    'varname': 'v'})

ss.setup()

msg_ltb = f"LTB case: {ss.files.case.split('/')[-1]}.\n"
msg_htb = f"HTB is set to LTB load {pq_htb}, connected to LTB bus {bus_htb}."
logger.warning(msg_ltb + msg_htb)
logger.warning("LTB setup completed.")

rows = np.ceil((cs_config['t_total'] - cs_config['ti']) / cs_config['t_step'])
cs_num = -1 * np.ones((int(rows), len(cs_col)))

const_freq = 2 * np.pi * ss.config.freq  # constant to calculate bus angle
flag_init = True  # Flag to indicate the very first iteration
flag_tc0 = True  # Flag to record `tc0`

flag_datar = False  # Flag to indicate if data read is successful
flag_dataw = False  # Flag to indicate if data write is successful

flag_base = True  # Flag t0 update counter base

# --- system initial conditions ---

a0 = ss.Bus.get(idx=bus_slack, src='a', attr='v')  # initial slack bus angle
p0 = ss.PQ.get(idx=pq_htb, src='p0', attr='v')  # initial HTB bus active power
q0 = ss.PQ.get(idx=pq_htb, src='q0', attr='v')  # initial HTB bus reactive power

ss.TDS.config.no_tqdm = 1  # turn off tqdm progress bar
ss.TDS.config.criteria = 0  # turn off stability criteria

# set constant power load
ss.PQ.config.p2p = 1
ss.PQ.config.q2q = 1
ss.PQ.config.p2z = 0
ss.PQ.config.q2z = 0
ss.PQ.pq2z = 0


Working directory: "/home/jwang175/hlcosim/code"
> Loaded config from file "andes.rc"
> Loaded generated Python code in "/home/jwang175/.andes/pycode".
Parsing input file "/home/jwang175/hlcosim/code/case/ieee39_htb.xlsx"...
Input file parsed in 0.4154 seconds.
System internal structure set up in 0.0516 seconds.
LTB case: ieee39_htb.xlsx.
HTB is set to LTB load PQ_2, connected to LTB bus 4.
LTB setup completed.


In [6]:
ss.PFlow.run()

-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: Off
   Sparse solver: KLU
 Solution method: NR method
Power flow initialized in 0.0107 seconds.
0: |F(x)| = 9.601680411
1: |F(x)| = 3.4980851
2: |F(x)| = 0.1830556615
3: |F(x)| = 0.0007295596655
4: |F(x)| = 1.673593752e-08
Converged in 5 iterations in 0.0161 seconds.


True

In [7]:
for end_time in np.arange(0, 20, 0.05):  # t_total
    # run TDS
    if end_time == 0:
        continue
    else:
        # run TDS
        ss.TDS.config.tf = end_time
        ss.TDS.run()


-> Time Domain Simulation Summary:
Sparse Solver: KLU
Simulation time: 0.0-0.05 s.
Fixed step size: h=33.33 ms. Shrink if not converged.
GENROU (vf range) out of typical lower limit.

   idx     | values | limit
-----------+--------+------
 GENROU_10 | 0.980  | 1    


GENROU (xl <= xd2) out of typical upper limit.

   idx     | values | limit
-----------+--------+------
 GENROU_1  | 0.014  | 0.001
 GENROU_3  | 0.018  | 0.003
 GENROU_4  | 0.013  | 0.001
 GENROU_5  | 0.014  | 0.001
 GENROU_7  | 0.015  | 0.002
 GENROU_8  | 0.015  | 0.006
 GENROU_9  | 0.009  | 0.001
 GENROU_10 | 0.013  | 0.000


Dime not installed. Set System config `dime_enabled` to `0` to suppress warning.
Initialization for dynamics completed in 0.1029 seconds.
Initialization was successful.
Simulation to t=0.05 sec completed in 0.0155 seconds.
Simulation to t=0.10 sec completed in 0.0151 seconds.
Simulation to t=0.15 sec completed in 0.0062 seconds.
Simulation to t=0.20 sec completed in 0.0062 seconds.
Simulation to 